In [1]:
from dotenv import load_dotenv,find_dotenv
import os

load_dotenv(find_dotenv('C:/Code_Apps/Learn-Generative-AI/03_chatgpt/.env'))

api_key = os.environ.get('OPENAI_API_KEY')

![ALT TEXT](https://raw.githubusercontent.com/panaverse/learn-generative-ai/788f968387b0ad38bc379c3a4718400e8e42948d/03_chatgpt/05_function_calling/first.png)

In [3]:
# Function defining for Function Calling
import json

def get_local_weather(city:str,unit:str="fahrenheit")->str:
    """Giving local cities temperature"""
    if "lahore" in city.lower():
        return json.dumps({"city":"Lahore","temperature":"51","unit":"fahrenheit"})
    elif "karachi" in city.lower():
        return json.dumps({"city":"Karachi","temperature":"24","unit":"celsius"})
    elif "gilgit" in city.lower():
        return json.dumps({"city":"Gilgit","temperature":"24","unit":"fahrenheit"})
    else:
        return json.dumps({"city":city,"temperature":"Insufficient Data"})


In [11]:
from openai import OpenAI
from openai.types.chat.chat_completion import ChatCompletion,ChatCompletionMessage

client = OpenAI()

def chat_completion(prompt):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_local_weather",
                "description": "Get the current weather in a given city of Pakistan",
                "parameters": {
                    "type": "object",
                    "properties":{
                        "city": {
                            "type": "string",
                            "description": "The name of the city to get the weather for e.g Lahore , Peshawar"
                        },
                        "unit":{
                            "type": "string",
                            "description": "The unit of temperature to return the weather in e.g Celsius , Fahrenheit",
                            "enum": ["Celsius", "Fahrenheit"] 
                        }
                    },
                    "required": ["city"]
                },
            }
        }
    ]

    messages = [{
            "role": "user",
            "content": prompt,
            }]

    response : ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools = tools,
        tool_choice= "auto"
    )

    display("* First Response: ", dict(response))
    print()
    print()


    response_message: ChatCompletionMessage = response.choices[0].message
    messages.append(response_message)
    display("Appended Messages: ", list(messages))
    print()
    print()


    toolcalls = response_message.tool_calls
    display("* First Response Tool calls: ", list(toolcalls))
    print()
    print()



    if toolcalls:
        available_functions = {
            'get_local_weather': get_local_weather
        }
        print("Toolcalls detail: ")
        

        for tool in toolcalls:
            function_name = tool.function.name
            function_parameters = json.loads(tool.function.arguments) # json.loads->string to json
            function_to_call = available_functions[function_name]
            function_response = function_to_call(
                city=function_parameters["city"],
                unit=function_parameters["unit"]
            ) # Function Calling is happening (Orchestration)

            messages.append({
                "tool_call_id": tool.id,
                "name": function_name,
                "role": "tool",
                "content": function_response
            })
            
        display("* Second Request Messages: ", list(messages))

        finalResponse = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )

        display("* Final Response: ", dict(finalResponse))
        print()
        print()
        return finalResponse.choices[0].message.content

In [14]:
print(chat_completion("What is the weather in Karachi, and in San Francisco, And in Lahore also? All are given line-by-line, And also let me know if the given city belong to Pakistan or any other Country?"))

'* First Response: '

{'id': 'chatcmpl-8mvtaodMvccMvvgqWLboVUngjMjbV',
 'choices': [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_CjL2oRuYKuI17c0n3j2PcLmR', function=Function(arguments='{"city": "Karachi", "unit": "Celsius"}', name='get_local_weather'), type='function'), ChatCompletionMessageToolCall(id='call_nV3MLN5YyO3qx3T7Zg6OJRxi', function=Function(arguments='{"city": "San Francisco", "unit": "Celsius"}', name='get_local_weather'), type='function'), ChatCompletionMessageToolCall(id='call_gamxK693U512rAZ9RweqGmuv', function=Function(arguments='{"city": "Lahore", "unit": "Celsius"}', name='get_local_weather'), type='function')]))],
 'created': 1706672986,
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_b57c83dd65',
 'usage': CompletionUsage(completion_tokens=79, prompt_tokens=139, total_tokens=218)}

'Appended Messages: '

[{'role': 'user',
  'content': 'What is the weather in Karachi, and in San Francisco, And in Lahore also? All are given line-by-line, And also let me know if the given city belong to Pakistan or any other Country?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_CjL2oRuYKuI17c0n3j2PcLmR', function=Function(arguments='{"city": "Karachi", "unit": "Celsius"}', name='get_local_weather'), type='function'), ChatCompletionMessageToolCall(id='call_nV3MLN5YyO3qx3T7Zg6OJRxi', function=Function(arguments='{"city": "San Francisco", "unit": "Celsius"}', name='get_local_weather'), type='function'), ChatCompletionMessageToolCall(id='call_gamxK693U512rAZ9RweqGmuv', function=Function(arguments='{"city": "Lahore", "unit": "Celsius"}', name='get_local_weather'), type='function')])]

'* First Response Tool calls: '

...

'* Second Request Messages: '

[{'role': 'user',
  'content': 'What is the weather in Karachi, and in San Francisco, And in Lahore also? All are given line-by-line, And also let me know if the given city belong to Pakistan or any other Country?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_CjL2oRuYKuI17c0n3j2PcLmR', function=Function(arguments='{"city": "Karachi", "unit": "Celsius"}', name='get_local_weather'), type='function'), ChatCompletionMessageToolCall(id='call_nV3MLN5YyO3qx3T7Zg6OJRxi', function=Function(arguments='{"city": "San Francisco", "unit": "Celsius"}', name='get_local_weather'), type='function'), ChatCompletionMessageToolCall(id='call_gamxK693U512rAZ9RweqGmuv', function=Function(arguments='{"city": "Lahore", "unit": "Celsius"}', name='get_local_weather'), type='function')]),
 {'tool_call_id': 'call_CjL2oRuYKuI17c0n3j2PcLmR',
  'name': 'get_local_weather',
  'role': 'tool',
  'content': '{"city": "Karachi", "temperature

'* Final Response: '

{'id': 'chatcmpl-8mvtdPOJncnAaUksD3vZOvg3Yot8e',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The weather in Karachi is 24°C. Karachi is in Pakistan.\n\nThe weather in San Francisco is not available.\n\nThe weather in Lahore is 51°F. Lahore is in Pakistan.', role='assistant', function_call=None, tool_calls=None))],
 'created': 1706672989,
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_b57c83dd65',
 'usage': CompletionUsage(completion_tokens=37, prompt_tokens=195, total_tokens=232)}



The weather in Karachi is 24°C. Karachi is in Pakistan.

The weather in San Francisco is not available.

The weather in Lahore is 51°F. Lahore is in Pakistan.
